## 일베 댓글+ 캐글 데이터로 fast-text 임베딩 하기

In [2]:
import pandas as pd
import numpy as np
import re

from gensim.models import FastText
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [32]:
df = pd.read_csv('./dataset/비속어댓글총합_특수문자,null값제거o.csv')
df

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
190576,드라마도 완전 재밌고 배우들도 멋있어서,0
190577,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
190578,케석대 어깨 올라간거봐라,1
190579,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


# 자모 분류, fast text 는 git 참고 
https://github.com/smothly/bad-word-detection/blob/master/FastTextVocab.ipynb

In [25]:

CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
TOTAL = CHOSUNGS + JOONGSUNGS + JONGSUNGS

# 자모분리
def jamo_split(word, end_char="_"):
    
    result = []
    
    for char in word:
        
        character_code = ord(char)
        
        if 0xD7A3 < character_code or character_code < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((character_code - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((character_code - 0xAC00) / 28) % 21)
        jongsung_index = int((character_code - 0xAC00) % 28)
        
        chosung = CHOSUNGS[chosung_index]
        joongsung = JOONGSUNGS[joongsung_index]
        jongsung = JONGSUNGS[jongsung_index]
        
        # 종성 범위 밖에 있는 것들은 end_char로 메꿔준다.
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)


In [33]:
# 각 문장을 jamo_split하여 저장
#from JamoSplit import jamo_split, jamo_combine
df.loc[:,'댓글'] = df.loc[:,'댓글'].apply(lambda x: jamo_split(x))
df.loc[:,'댓글'] = df.loc[:,'댓글'].apply(lambda x: x.split(" "))

In [35]:
df.head(15)

,댓글,비속어여부
0,"[ㅈㅘ_ㅂㅐ_, ㄲㅏ_ㄴㅡㄴㄱㅓㄴ]",1
1,"[ㅈㅣㅂㅇㅔ_, ㄹㅗㅇ, ㅍㅐ_ㄷㅣㅇㅁㅏㄴ, ㅅㅔ_, ㄱㅐ_ㄷㅏ_, 10ㄴㅕㄴ, ㄷ...",0
2,"[ㄱㅐ_ㅅㅗ_ㄹㅣ_ㅇㅑ_, ㄴㅣ_ㄱㅏ_, ㅃㅏㄹㄱㅐㅇㅇㅣ_ㄹㅡㄹ, ㅇㅗㅇㅎㅗ_ㅎㅏ_...",1
3,"[ㅅㅔ_ㅌㅏㄱㅇㅣ_ㄹㅏ_ㄱㅗ_, ㅂㅘ_ㄷㅗ_, ㄷㅚㄴㄷㅏ_]",0
4,"[ㅇㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_, ㅊㅗ_ㄷㅣㅇㄷㅗ_, ㅇㅏ_ㄴㅣ_ㄱㅗ_]",1
5,"[731ㅂㅜ_ㄷㅐ_ㅇㅢ_, ㅎㅜ_ㅇㅖ_ㄹㅏ_, ㄱㅡ_ㄹㅓㄴㅈㅣ_, ㄱㅏ_ㅎㅏㄱㅈㅓㄱ...",1
6,[ㅈㅐ_ㅇㅏㅇㅇㅣ_ㅎㅏㄴㄱㅓㄴㅎㅐㅅㄴㅗ_],1
7,"[ㄱㅡㄹㅆㅡㄴㅇㅣ_, ㅇㅘ_ㄲㅜ_, ㅅㅡㅇㄹㅣ_ㅇㅔ_, ㅂㅣ_ㅎㅏ_ㅁㅕㄴ, ㅂㅏㅇㅅ...",1
8,"[ㅁㅏ_, ㅆㅣ_ㅂㅏㄹㄹㅕㄴ, ㅇㅏ_, ㅁㅕㅊㅍㅕㅇㅇㅣ_ㄱㅗ_, ㅁㅐㅅㄱㅐ_ㄷㅡ_ㄱ...",1
9,"[ㅇㅡㄴㅎㅐㅇㅇㅔ_, ㄷㅐ_ㅊㅜㄹ, ㅅㅏㅇㄷㅏㅁ, ㅂㅏㄷㅇㅡ_ㄹㅓ_, ㄱㅏ_ㅂㅗ_ㅁ...",0


In [37]:
# dataframe -> list로 변환
sentence_list = list(df['댓글'])
len(sentence_list)

190581

In [38]:
# fasttext 인풋 완성!
sentence_list[:10]

[['ㅈㅘ_ㅂㅐ_', 'ㄲㅏ_ㄴㅡㄴㄱㅓㄴ'],
 ['ㅈㅣㅂㅇㅔ_',
  'ㄹㅗㅇ',
  'ㅍㅐ_ㄷㅣㅇㅁㅏㄴ',
  'ㅅㅔ_',
  'ㄱㅐ_ㄷㅏ_',
  '10ㄴㅕㄴ',
  'ㄷㅓ_',
  'ㅇㅣㅂㅇㅓ_ㅇㅑ_ㅈㅣ_'],
 ['ㄱㅐ_ㅅㅗ_ㄹㅣ_ㅇㅑ_',
  'ㄴㅣ_ㄱㅏ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅣ_ㄹㅡㄹ',
  'ㅇㅗㅇㅎㅗ_ㅎㅏ_ㄱㅗ_',
  'ㄷㅡ_ㄹㅜ_ㅋㅣㅇㅇㅡㄹ',
  'ㅈㅣㅅㅇㅣ_ㄹㅏ_ㄱㅗ_',
  'ㅁㅏㄹㅁㅗㅅㅎㅐ_ㅅㅓ_',
  'ㅃㅣ_ㅈㅣㄴㄱㅓ_ㅇㅑ_',
  'ㅃㅏㄹㄱㅐㅇㅇㅏ_'],
 ['ㅅㅔ_ㅌㅏㄱㅇㅣ_ㄹㅏ_ㄱㅗ_', 'ㅂㅘ_ㄷㅗ_', 'ㄷㅚㄴㄷㅏ_'],
 ['ㅇㅐ_ㅅㅐ_ㄲㅣ_ㄱㅏ_', 'ㅊㅗ_ㄷㅣㅇㄷㅗ_', 'ㅇㅏ_ㄴㅣ_ㄱㅗ_'],
 ['731ㅂㅜ_ㄷㅐ_ㅇㅢ_',
  'ㅎㅜ_ㅇㅖ_ㄹㅏ_',
  'ㄱㅡ_ㄹㅓㄴㅈㅣ_',
  'ㄱㅏ_ㅎㅏㄱㅈㅓㄱㅇㅣㄴ',
  'ㅇㅏ_ㅇㅣ_ㄷㅣ_ㅇㅓ_ㄴㅡㄴ',
  'ㅅㅔ_ㄱㅖ_ㅊㅚ_ㄱㅗ_ㅇㅣㅁ',
  'ㅇㅣ_ㄹㅐ_ㅅㅓ_',
  'ㅇㅐ_ㄱㅛ_ㅁㅏㄴ',
  'ㄸㅓㄹㅇㅓ_ㄷㅗ_',
  'ㄷㅗㄴ',
  'ㅂㅓㄹㄹㅣ_ㄴㅡㄴ',
  'ㅎㅏㄴㄱㅜㄱㅇㅔ_',
  'ㄱㅣ_ㄹㅡㄹ',
  'ㅆㅓ_ㅅㅓ_',
  'ㅈㅣㄴㅊㅜㄹㅎㅏ_ㄹㅕ_ㄱㅗ_',
  'ㅎㅏ_ㅈㅣ_ㅈㅗ_ㅅㅔㄴㄴㅏㅁㅈㅏ_ㄷㅡㄹㅇㅡㄴ',
  'ㄸㅗ_',
  'ㅇㅣ_ㅃㅡㄴㅇㅕ_ㅈㅏ_ㅁㅏㄴ',
  'ㅂㅗ_ㅁㅕㄴ',
  'ㅅㅏ_ㅈㅗㄱㅇㅡㄹ',
  'ㅁㅗㅅㅆㅡ_ㅁㅕ_',
  'ㄱㅗㅇㅈㅜ_ㄷㅐ_ㅈㅓㅂㅎㅐ_ㅈㅜ_ㄴㅡㄴ',
  'ㄴㅗㅁㄷㅡㄹㅇㅣ_ㄴㅣ_'],
 ['ㅈㅐ_ㅇㅏㅇㅇㅣ_ㅎㅏㄴㄱㅓㄴㅎㅐㅅㄴㅗ_'],
 ['ㄱㅡㄹㅆㅡㄴㅇㅣ_',
  'ㅇㅘ_ㄲㅜ_',
  'ㅅㅡㅇㄹㅣ_ㅇㅔ_',
  'ㅂㅣ_ㅎㅏ_ㅁㅕㄴ',
  'ㅂㅏㅇㅅㅏ_ㄴㅡㅇ',
  'ㅍㅣ_ㅍㅗㄱ',
  'ㅇㅝㄴㅅㅜㅇㅇㅣ_',
  'ㅇㅣㄹㄷㅡㅅ'],
 ['ㅁㅏ_',
  'ㅆㅣ_ㅂㅏㄹㄹㅕㄴ',
  'ㅇㅏ_',
  'ㅁㅕㅊㅍㅕㅇㅇㅣ_ㄱㅗ_',
  'ㅁㅐㅅㄱㅐ_ㄷㅡ_ㄱㅏㅆㄴㅗ_',
  'ㄴㅣ_',
  'ㄷㅐ_ㅎㅏ_ㅇㅣ_ㅎㅐㅁㅎㅏ_ㄱㅗ_',
  'ㅎㅐ_ㅂ

### FastText WordEmbedding

In [70]:
# 임베딩 차원: 100
# window size: 좌우 2단어 비속어는 좌우단어와 별로 연관이 없다고 판단...
# min_count: 최소 3번 등장한 단어들
# workers: -1 전부!!
# sg: skipgram이 더 성능이 좋기 때문
# min_n max_n : n-gram단위인데 한글자가 3글자라 최소 자모3개부터 최대 6개까지 ngram하기로 하였다. 1글자 ~ 2글자
# iter: 반복횟수 10

model = FastText(sentence_list, size=100, window=2, min_count=3, workers=4, sg=1, min_n=3, max_n=6, iter=10)

In [71]:
# 총 62105개 단어로 vocab이 만들어졌다.
# len(model.wv)
len(model.wv.vocab)

62105

In [39]:
# 연도 같은 경우는 연도와 비슷한 단어들이 나온다.
model.wv.most_similar(jamo_split("2018년"))

NameError: name 'model' is not defined

In [73]:
# 욕설 같은 경우는 비슷한 형태의 욕설이 나온다.
model.wv.most_similar(jamo_split("개새끼"))

[('ㅈㅟ_ㅅㅐ_ㄲㅣ_', 0.9755383729934692),
 ('ㅊㅣ_ㅁㅐ_ㅅㅐ_ㄲㅣ_', 0.9718326330184937),
 ('ㅇㅐ_ㅅㅐ_ㄲㅣ_', 0.9717241525650024),
 ('ㄱㅐ_ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9717193841934204),
 ('ㅅㅣㅂㅅㅐ_ㄲㅣ_', 0.9686024785041809),
 ('ㄱㅐ_ㄷㅗㄱㅅㅐ_ㄲㅣ_', 0.96709805727005),
 ('ㅉㅏㅇㄲㅐ_ㅅㅐ_ㄲㅣ_', 0.9652191400527954),
 ('ㅊㅣ_ㅌㅏ_ㅅㅐ_ㄲㅣ_', 0.9636259078979492),
 ('ㅆㅣㅂㅅㅐ_ㄲㅣ_', 0.9634711146354675),
 ('ㅁㅜㄴㅅㅐ_ㄲㅣ_', 0.9631229043006897)]

In [74]:
model.wv.most_similar(jamo_split("시발"))

[('ㅁㅝㄴㅅㅣ_ㅂㅏㄹ', 0.9581457376480103),
 ('ㅆㅣ_ㅂㅏㄹ', 0.9515796899795532),
 ('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9480047225952148),
 ('ㅅㅣ_ㅂㅏㅋ', 0.9453717470169067),
 ('ㅇㅘ_ㅆㅣ_ㅂㅏㄹ', 0.9367014169692993),
 ('ㅅㅣ_ㅂㅏㄹㅇㅘ_', 0.932921290397644),
 ('ㅇㅘ_ㅅㅣ_ㅂㅏㄹ', 0.9328004121780396),
 ('ㅅㅣ_ㅂㅏㄹㄲㅓ_', 0.9291257858276367),
 ('ㅅㅣ_ㅂㅏㄹㄴㅗㅁ', 0.9290575981140137),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴ', 0.9143559336662292)]

In [75]:
model.wv.most_similar(jamo_split("시바"))

[('ㅅㅣ_ㅂㅏㅋ', 0.9333863258361816),
 ('ㅅㅣ_ㅂㅏㄹㄹㅓㅁㅇㅏ_', 0.9325875639915466),
 ('ㅅㅣ_ㅂㅏㄹㄹㅗㅁㅇㅏ_', 0.9319903254508972),
 ('ㅅㅣ_ㅂㅏㄹㄴㅓㅁ', 0.9314566850662231),
 ('ㅅㅣ_ㅂㅏㄹㄲㅓ_', 0.9202462434768677),
 ('ㅅㅣ_ㅂㅏㄹㅇㅏ_', 0.9190757870674133),
 ('ㅅㅣ_ㅂㅏㄹㄹㅕㄴㅇㅏ_', 0.9178509712219238),
 ('ㅅㅣ_ㅂㅏㄹㄴㅗㅁ', 0.9153333902359009),
 ('ㅅㅣ_ㅂㅏㄹㄴㅕㄴㅇㅏ_', 0.9124631881713867),
 ('ㅆㅣ_ㅂㅏ_', 0.909488320350647)]

In [76]:
model.wv.most_similar(jamo_split("미친"))

[('ㅁㅣ_ㅊㅣㄴㄴㅓㅁ', 0.9066715836524963),
 ('ㅁㅣ_ㅊㅣㄴㅈㅣㅅ', 0.8714250922203064),
 ('ㅁㅣ_ㅊㅣㄴㄴㅗㅁ', 0.8459020256996155),
 ('ㄱㅐ_ㅁㅣ_ㅊㅣㄴ', 0.8347834348678589),
 ('ㅁㅣ_ㅊㅣㄹ', 0.8251173496246338),
 ('ㅁㅣ_ㅊㅣㄴㄴㅕㄴ', 0.8197712898254395),
 ('ㅂㅣ_ㅊㅣㄴ', 0.8066060543060303),
 ('ㅁㅣ_ㅊㅕ_', 0.7836000919342041),
 ('ㅁㅣ_ㅊㅣㄴㄴㅗㅁㅇㅏ_', 0.782168984413147),
 ('ㅈㅣ_ㅊㅣㄴ', 0.7717003226280212)]

In [77]:
model.wv.most_similar(jamo_split("존나"))

[('ㅈㅗㄴㄴㅏ_ㅍㅐㅁ', 0.9640662670135498),
 ('ㅅㅗㄴㄴㅏ_', 0.9248775839805603),
 ('ㅈㅗ_ㅇㅗㄴㄴㅏ_', 0.906119704246521),
 ('ㅈㅗㄴㄴㅏ_ㅎㅏㅁ', 0.8897804021835327),
 ('ㅈㅗㄴㄴㅏ_ㅇㅜㅅㅈㅏ_', 0.8823866844177246),
 ('ㅈㅛㄴㄴㅏ_', 0.8606939911842346),
 ('ㅈㅝㄴㄴㅏ_', 0.8542730808258057),
 ('ㅈㅗㄴㄴㅏ_ㅁㅏㄶㄴㅔ_', 0.852696418762207),
 ('ㅈㅗㄴㄴㅏ_ㅅㅣㅀㅇㅡㅁ', 0.8515698313713074),
 ('ㅇㅛㄴㄴㅏ_', 0.8508866429328918)]

In [78]:
model.wv.most_similar(jamo_split("지랄"))

[('ㅈㅣ_ㄹㅏㄹㅂㅕㅇ', 0.9685596227645874),
 ('ㅈㅣ_ㄹㅏㄹㅈㅗㅁ', 0.9577693939208984),
 ('ㅈㅣ_ㄹㅏㄹㅂㅏㄹㄱㅘㅇ', 0.9528383016586304),
 ('ㅈㅣ_ㄹㅏㄹㅁㅏ_', 0.9420679211616516),
 ('ㅈㅗㅈㅈㅣ_ㄹㅏㄹ', 0.9324055910110474),
 ('ㅈㅣ_ㄹㅏㄹㅇㅣㅁ', 0.9312236309051514),
 ('ㅂㅕㄹㅈㅣ_ㄹㅏㄹ', 0.9237711429595947),
 ('ㅈㅣ_ㄹㅏㄹㅎㅏㅁ', 0.9230659604072571),
 ('ㅈㅣ_ㄹㅏㄹㅎㅏㄹ', 0.9047238826751709),
 ('ㄱㅐ_ㅈㅣ_ㄹㅏㄹ', 0.9012449979782104)]

In [83]:
model.wv.most_similar(jamo_split("쓰레기"))

[('ㅆㅣㅂㅆㅡ_ㄹㅔ_ㄱㅣ_', 0.9678969979286194),
 ('ㅈㅗㅈㅆㅡ_ㄹㅔ_ㄱㅣ_', 0.9603729248046875),
 ('ㄱㅐ_ㅆㅡ_ㄹㅔ_ㄱㅣ_', 0.954119086265564),
 ('ㅆㅡ_ㄹㅔ_ㄱㅣ_ㅌㅗㅇ', 0.9291515350341797),
 ('ㄱㅐ_ㅆㅣㅂㅆㅡ_ㄹㅔ_ㄱㅣ_', 0.9211405515670776),
 ('ㅇㅣㄴㄱㅏㄴㅆㅡ_ㄹㅔ_ㄱㅣ_', 0.9059302806854248),
 ('ㅆㅡ_ㄹㅔ_ㄱㅣ_ㄴㅣ_', 0.9002506136894226),
 ('ㄱㅣ_ㄹㅔ_ㄱㅣ_', 0.8951807022094727),
 ('ㅆㅡ_ㄹㅔ_ㄱㅣ_ㅇㅣㅁ', 0.8636864423751831),
 ('ㅆㅡ_ㄹㅐ_ㄱㅣ_', 0.8620851039886475)]

In [84]:
# embedding model 저장
model.save("./festtext_embedded_1.model")

### ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [27]:
# fasttext 모델 불러오기

from gensim.models import FastText

embedded_model = FastText.load("./festtext_embedded_1.model")

In [86]:
# 각 단어를 벡터화 시켜주는 과정 3 x 100(embedding dimension) 
df['댓글'] = df['댓글'].apply(lambda x: [embedded_model[_] for _ in x])

/tmp/ipykernel_31149/1170649138.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  df['댓글'] = df['댓글'].apply(lambda x: [embedded_model[_] for _ in x])


In [87]:
df.head()

,댓글,비속어여부
0,"[[-0.6416038, 0.39122328, -0.45883512, -0.1057...",1
1,"[[0.060996383, -0.039162956, 0.9557268, 1.0635...",0
2,"[[-0.31791896, 1.0826435, -0.18621485, 0.75556...",1
3,"[[-0.59272283, 0.58092606, 0.6686394, 0.181487...",0
4,"[[-0.2995909, 0.94177955, -0.5402276, 0.117031...",1


In [88]:
# 벡터화된 데이터로 저장
df.to_json("./fasttext_vectorized_1.json")

## 패딩하기

In [3]:
df2 = pd.read_json("./fasttext_vectorized_1.json")
df2.columns = ["댓글", "비속어여부"]
df2.head()

,댓글,비속어여부
0,"[[-0.6416038275, 0.39122328160000003, -0.45883...",1
1,"[[0.0609963834, -0.0391629562, 0.9557268023000...",0
2,"[[-0.3179189563, 1.0826435089, -0.1862148494, ...",1
3,"[[-0.5927228332000001, 0.5809260607, 0.6686394...",0
4,"[[-0.2995908856, 0.9417795539, -0.540227592, 0...",1


In [4]:
# 상위 20개 문장의 단어 개수 살펴보기
for i in range(0,20):
    print(len(df2.iloc[i,0]))

2
8
9
3
3
23
1
8
8
9
7
13
5
7
8
4
8
19
4
2


In [11]:
print('문장의 최대 단어길이 :',max(len(review) for review in df2['댓글']))
print('문장의 평균 단어길이 :',sum(map(len, df2['댓글']))/len(df2['댓글']))

# 한 문장의 최대 단어의 개수는 312개, 평균 단어의 개수는 8개

문장의 최대 단어길이 : 312
문장의 평균 단어길이 : 8.244289829521305


In [5]:
def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 문장들 중 단어개수가 %s 이하인 문장의 비율: %s'%(max_len, (count / len(nested_list))*100))
    
max_len = 30
below_threshold_len(max_len, df2['댓글'])

# 97 % 가 30 단어 이하이기 때문에 최대단어수는 30으로 패딩해주는것이 좋을것같다.

전체 문장들 중 단어개수가 30 이하인 문장의 비율: 97.1733803474638


In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df2['댓글'], df2['비속어여부'] , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

152464 152464 38117 38117


In [7]:
df2['댓글'].head(10)

0    [[-0.6416038275, 0.39122328160000003, -0.45883...
1    [[0.0609963834, -0.0391629562, 0.9557268023000...
2    [[-0.3179189563, 1.0826435089, -0.1862148494, ...
3    [[-0.5927228332000001, 0.5809260607, 0.6686394...
4    [[-0.2995908856, 0.9417795539, -0.540227592, 0...
5    [[-0.1525025368, -0.38018080590000003, -0.2431...
6    [[-0.5504084826, 0.7977899313, -0.079168982800...
7    [[-0.9684337378000001, 1.1426610947, -0.196303...
8    [[-0.3493541181, 0.7854241133000001, -1.033611...
9    [[-0.3350792825, 0.09681017700000001, 0.822802...
Name: 댓글, dtype: object

In [8]:
train_x

46726     [[-0.4425993264, -0.0899327248, 0.0366247185, ...
94165     [[-0.5336186886000001, 0.6592813134000001, -0....
136954    [[-1.1767054796, -0.0874177217, 1.0539677143, ...
183719    [[-0.1866511256, 0.4419595897, 0.0912217647, -...
133146    [[0.24270738660000002, 0.8839552999, -0.201448...
                                ...                        
152315    [[-0.42085075380000003, -0.15677140650000002, ...
176963    [[0.2120446712, -0.2478029579, -0.4419538975, ...
117952    [[-0.7531749606, 0.1324819475, -0.2797733247, ...
173685    [[-0.6214374304, -0.019838854700000002, -0.857...
43567     [[-0.10557022690000001, 0.7503887415, -0.19460...
Name: 댓글, Length: 152464, dtype: object

In [9]:
train_y

46726     1
94165     1
136954    0
183719    0
133146    0
         ..
152315    0
176963    0
117952    1
173685    0
43567     1
Name: 비속어여부, Length: 152464, dtype: int64

In [ ]:
# a=[]
# for i in train_x:
#     a.append(np.array(train_x))
# a

In [13]:
# y array형식으로 변경
y_train = np.array(train_y)
y_test = np.array(test_y)

In [14]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(train_x, maxlen=max_len)
X_test = pad_sequences(test_x, maxlen=max_len)

In [56]:
X_train[:]

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  1, ..., -1,  0,  0],
        [ 0,  1,  0, ...,  0,  0,  0],
        [ 0,  1, -1, ...,  1,  0

In [21]:
print(X_test.shape)
print(X_train.shape)

(38117, 30, 100)
(152464, 30, 100)


In [22]:
print(y_test.shape)
print(y_train.shape)

(38117,)
(152464,)


In [55]:
y_test[:]

array([0, 0, 1, ..., 0, 1, 1])

### LSTM 학습하기 

In [28]:
from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(LSTM(units=1, input_shape=(30, 100)))
model.add(Dense(1, activation='sigmoid'))


# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
# mc = ModelCheckpoint('FT_best_model_v0.0.1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)


model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

2022-05-11 04:43:10.655291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-11 04:43:10.889755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-11 04:43:10.891584: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-11 04:43:10.986634: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-11 04:43:11.078489: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-11 04:43:11.083474: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

Epoch 1/10
4765/4765 [==============================] - 29s 6ms/step - loss: 0.3397 - accuracy: 0.8621 - val_loss: 0.2899 - val_accuracy: 0.8788
Epoch 2/10
4765/4765 [==============================] - 29s 6ms/step - loss: 0.2817 - accuracy: 0.8808 - val_loss: 0.2839 - val_accuracy: 0.8785
Epoch 3/10
4765/4765 [==============================] - 28s 6ms/step - loss: 0.2774 - accuracy: 0.8817 - val_loss: 0.2829 - val_accuracy: 0.8792
Epoch 4/10
4765/4765 [==============================] - 28s 6ms/step - loss: 0.2763 - accuracy: 0.8820 - val_loss: 0.2828 - val_accuracy: 0.8788
Epoch 5/10
4765/4765 [==============================] - 28s 6ms/step - loss: 0.2756 - accuracy: 0.8820 - val_loss: 0.2820 - val_accuracy: 0.8792
Epoch 6/10
4765/4765 [==============================] - 28s 6ms/step - loss: 0.2753 - accuracy: 0.8817 - val_loss: 0.2835 - val_accuracy: 0.8780
Epoch 7/10
4765/4765 [==============================] - 28s 6ms/step - loss: 0.2751 - accuracy: 0.8824 - val_loss: 0.2814 - val_ac

In [31]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1])) # model.evaluate(X_test, y_test)
# x와 y 의 test 에 대한 정확도 출력

1192/1192 [==============================] - 2s 2ms/step - loss: 0.2820 - accuracy: 0.8792

 테스트 정확도: 0.8792


In [32]:
model.save("./fasttext_LSTM1.h5")

In [28]:
# model.save("./fasttext_model")

2022-05-10 09:01:21.980801: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./fasttext_model/assets


In [36]:
lstm_model = load_model('./fasttext_LSTM1.h5')

2022-05-11 09:29:27.588928: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-11 09:29:27.732822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-11 09:29:27.734832: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-11 09:29:27.829004: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-11 09:29:27.930799: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-11 09:29:27.935553: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

# 모델 사용해서 예측하기

### for index in range(len(test_word.split)) 일 경우 예측 - O

In [71]:
def test_result(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")


In [79]:
test_result("마라탕", embedded_model, lstm_model) # ㅠㅠ 

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/1192334688.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [83]:
test_result("지랄하지마", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/1192334688.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [84]:
test_result("나가서 놀고싶다", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/1192334688.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### for index in range(len(s)) 일 경우 예측 - O

In [80]:
def test_result2(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [85]:
test_result2("너무 어려워요", embedded_model, lstm_model)

/tmp/ipykernel_15844/3639305100.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있지 않습니다.


In [86]:
test_result2("마라탕", embedded_model, lstm_model) # ?? ㅠㅠ

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/3639305100.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [89]:
test_result2("졸라 시끄러워", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/3639305100.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### for index in range(len(test_word)) 일 경우 예측 - O

In [107]:
def test_result10(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [108]:
test_result10("졸라 시끄러워", embedded_model, lstm_model)

/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


In [109]:
test_result10("마라탕", embedded_model, lstm_model)

/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


In [111]:
test_result10("집에 갈꺼에요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [112]:
test_result10("집에 갈꺼에용", embedded_model, lstm_model) # 왜일까

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/88498840.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### 반대로 적용해보기 - X

In [88]:
def test_result3(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.9:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")

In [90]:
test_result3("졸라 시끄러워", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/3385605691.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [91]:
test_result3("돌체라떼 너무 맛있어요", embedded_model, lstm_model) # 반대로 하는거는 틀린것같음!!

/tmp/ipykernel_15844/3385605691.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


### range(len(test_word_split) 로 해보기 - X

In [95]:
def test_result4(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")

In [96]:
test_result4("돌체라떼 너무 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/2997558423.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [97]:
test_result4("돌체라떼 미친놈", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/2997558423.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


### 값을 낮춰보기 - O

In [99]:
def test_result5(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(s)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [100]:
test_result5("돌체라떼 너무 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/4085268211.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [101]:
test_result5("돌체라떼 미친놈", embedded_model, lstm_model)

/tmp/ipykernel_15844/4085268211.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


lstm 결과 : 비속어가 포함되어 있습니다.


### range(len(test_word_split))로 바꿔보기 - O

In [103]:
def test_result6(s, embedded_model, lstm_model):
    test_word = jamo_split(s)
    test_word_split = test_word.split()
    fast_vec = []
    for index in range(len(test_word_split)):
        if index < len(test_word_split):
            fast_vec.append(embedded_model[test_word_split[index]])
        else:
            fast_vec.append(np.array([0]*100))
    fast_vec = np.array(fast_vec)
    fast_vec=fast_vec.reshape(1, fast_vec.shape[0], fast_vec.shape[1])
    # 학습 데이터와 마찬가지로 3차원으로 크기 조절
    test_pre = lstm_model.predict_classes([fast_vec]) # 비속어 판별
    if test_pre[0][0] > 0.4:
        print("lstm 결과 : 비속어가 포함되어 있습니다.")
    else:
        print("lstm 결과 : 비속어가 포함되어 있지 않습니다.")

In [104]:
test_result6("돌체라떼 맛있어요", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있지 않습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [105]:
test_result6("돌체라떼 미친놈", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


In [106]:
test_result6("마라탕", embedded_model, lstm_model)

lstm 결과 : 비속어가 포함되어 있습니다.


/tmp/ipykernel_15844/155278071.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fast_vec.append(embedded_model[test_word_split[index]])


#### O = 어느정도 구분은 하지만 비속어가 아닌데도 비속어라고 판별하는 경우가 있음
#### 예를 들어 마라탕을 왜 비속어라고 분류하는지? 알 수 있는 방법이 없다..
#### len() 에 어떤것이 들어가야 하는지는 알아봐야한다. 